# Delta Practice

### File Read In

In [90]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

filepath4 = r'/Users/cartersocha/Downloads/instgramAccountCounts.xlsx'
igAccountDf = pd.read_excel(filepath4)

filepath5 = r'/Users/cartersocha/Downloads/redditCountTest.xlsx'
redditSubsDf = pd.read_excel(filepath5)

filepath6 = r'/Users/cartersocha/Downloads/redditCountFinal.txt'
#redditCommentsDf = pd.read_excel(filepath6)

filepath7 = r'/Users/cartersocha/Downloads/googleTvCount.xlsx'
googleTrendsDf = pd.read_excel(filepath7)

### Functions

In [91]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [92]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [93]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [94]:
def MergeDfs(mainDf, secondDf):

    merged = pd.merge(mainDf,secondDf, how='outer', on=['TvShow'])
    return merged

In [95]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf[columnName] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [96]:
def SumSocialMedia(sumDf,columnName):

    summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
    summarizedDf = pd.DataFrame(summarizedDf)

    return summarizedDf

### Data Transformations

##### Release Data

In [97]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=90)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [98]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'diffs')
instaHashDf['SocialMediaSource'] = 'InstagramHashtag'
instaHashDf['SocialMediaValue'] = instaHashDf['diffs']

iHashtagDf = SumSocialMedia(instaHashDf, 'TvShow')

iHashtagDf['SocialMediaValue'] = iHashtagDf['diffs']
iHashtagDf['SocialMediaSource'] = 'InstagramHashtag'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [99]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
dailyTweetDf['SocialMediaValue'] = dailyTweetDf['TweetCount'] + dailyTweetDf['RetweetCount']
dailyTweetDf['SocialMediaSource'] = 'Tweets'

iTweetCountDf = SumSocialMedia(dailyTweetDf, 'TvShow')
iTweetCountDf['SocialMediaSource'] = 'Tweets'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

redditCommentsDf = DateTimeConvert(redditCommentsDf,'runDate')

redCommentDf = RemoveData(releaseDf, redditCommentsDf)


iRedCommentDf = SumSocialMedia(redCommentDf, 'TvShow')
iRedCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']
iRedCommentDf['SocialMediaSource'] = 'RedditComments'

##### Instagram Account Data

In [100]:
igAccountDf = DateTimeConvert(igAccountDf,'RunDate')

igAccountDf = DiffMaker(igAccountDf,'IgAccountCounts','RunDate')


instaAccDf = RemoveData(releaseDf, igAccountDf)
instaAccDf = NegativeDiffs(instaAccDf, 'diffs')
instaAccDf['SocialMediaSource'] = 'InstagramAccount'
instaAccDf['SocialMediaValue'] = instaAccDf['diffs']

iAccountDf = SumSocialMedia(instaAccDf, 'TvShow')

iAccountDf['SocialMediaValue'] = iAccountDf['diffs']
iAccountDf['SocialMediaSource'] = 'InstagramAccount'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Subscriber Data

In [101]:
redditSubsDf = DateTimeConvert(redditSubsDf,'RunDate')

redditSubsDf = DiffMaker(redditSubsDf,'RedditSubs','RunDate')

redSubDf = RemoveData(releaseDf, redditSubsDf)
redSubDf = NegativeDiffs(redSubDf, 'diffs')
redSubDf['SocialMediaSource'] = 'RedditSubscribers'
redSubDf['SocialMediaValue'] = redSubDf['diffs']

iRedSubDf = SumSocialMedia(redSubDf, 'TvShow')
iRedSubDf['SocialMediaValue'] = iRedSubDf['diffs']
iRedSubDf['SocialMediaSource'] = 'RedditSubscribers'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Google Data

In [102]:
googleTrendsDf = DateTimeConvert(googleTrendsDf,'RunDate')

googDf = RemoveData(releaseDf, googleTrendsDf)
googDf = NegativeDiffs(googDf, 'GoogleValue')
googDf['SocialMediaSource'] = 'Google'
googDf['SocialMediaValue'] = googDf['GoogleValue']

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_95808/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Join the Data

##### Time Value Datasets

In [103]:
instaAccDf = instaAccDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
redSubDf = redSubDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
dailyTweetDf = dailyTweetDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
instaHashDf = instaHashDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
googDf = googDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]

In [110]:
uberDf2 = pd.concat([instaAccDf,redSubDf,dailyTweetDf,instaHashDf,googDf])

uberDf3 = MergeDfs(uberDf2, releaseDf)
uberDf3 = uberDf3[uberDf3['Cancel'] != 0]

##### Aggregation Datasets

In [105]:
igAccount = iAccountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
redditSub = iRedSubDf[['TvShow','SocialMediaValue','SocialMediaSource']]
tweetComments = iTweetCountDf[['TvShow','SocialMediaValue','SocialMediaSource']]
igHashtag = iHashtagDf[['TvShow','SocialMediaValue','SocialMediaSource']]

##### Dataset Merging

In [106]:
nuberData = pd.concat([igAccount,redditSub,tweetComments,igHashtag],axis=0)
nuberData.head()

,TvShow,SocialMediaValue,SocialMediaSource
0,ABlackLadySketchShow,7394.0,InstagramAccount
1,Archive81,89.0,InstagramAccount
2,Atlantafx,20436.0,InstagramAccount
3,BelAirPeacock,134549.0,InstagramAccount
4,Bridgerton,1146861.0,InstagramAccount


In [107]:
uberDf = MergeDfs(releaseDf, iHashtagDf)

uberDf = MergeDfs(uberDf, iTweetCountDf)

In [108]:
uberDf.head()

,TvShow,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,Cancel,90DayDate,HashtagValue,diffs,SocialMediaValue_x,SocialMediaSource_x,TweetCount,RetweetCount,SocialMediaValue_y,SocialMediaSource_y
0,ABlackLadySketchShow,HBOMax,6.0,3.0,Weekly,2022-04-08,NaN,2022-07-07,408351.0,464.0,464.0,InstagramHashtag,6700.0,6533.0,13233.0,Tweets
1,AboutLastNighthbo,HBOMax,8.0,1.0,Binge,2022-02-10,NaN,2022-05-11,334.0,1.0,1.0,InstagramHashtag,NaN,NaN,NaN,NaN
2,AdventureOfTheRing,HBOMax,8.0,1.0,Weekly,2022-02-10,0.0,2022-05-11,14492.0,2.0,2.0,InstagramHashtag,2213.0,4444.0,6657.0,Tweets
3,AllTheOldKnives,Prime,NaN,NaN,NaN,2022-04-08,0.0,2022-07-07,268196.0,2020.0,2020.0,InstagramHashtag,7416.0,5102.0,12518.0,Tweets
4,Archive81,Netflix,8.0,1.0,Binge,2022-01-14,NaN,2022-04-14,310066.0,982.0,982.0,InstagramHashtag,88380.0,48262.0,136642.0,Tweets


### Output - individuals & uber dataset

In [111]:
filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf3.to_csv(filepath2) 